## 9. Creating & managing asyncio tasks

`asyncio.create_task(coro)` schedules coroutine concurrently; returns **Task**.  `await task` waits; `asyncio.gather` aggregates many.  Use `asyncio.wait_for` for timeouts; unhandled exception in task propagates to loop.

```python
import asyncio, random
async def job(i):
    await asyncio.sleep(random.random())
    if i==2: raise ValueError('boom')
    return i

async def main():
    tasks=[asyncio.create_task(job(i)) for i in range(4)]
    try:
        results = await asyncio.gather(*tasks)
    except Exception as e:
        print('caught', e)
asyncio.run(main())
```

### Quick check

1. `asyncio.gather(..., return_exceptions=True)` will:
  a. raise first error   b. collect exceptions in results list

2. True / False Cancelling a task triggers `CancelledError` inside it.

<details><summary>Answer key</summary>

1. **b**.
2. **True**.

</details>

## 10. Mixing blocking and async code safely

Blocking call inside coroutine freezes loop.  Fix: `await loop.run_in_executor(None, blocking_fn, arg)` offloads to thread pool.  For CPU‑bound, prefer process pool.

```python
import asyncio, time
def blocking():
    time.sleep(1); return 'done'

async def main():
    loop = asyncio.get_running_loop()
    print(await loop.run_in_executor(None, blocking))
asyncio.run(main())
```

### Quick check

1. run_in_executor default uses:
  a. ProcessPool   b. ThreadPool

2. True / False CPU‑heavy tasks benefit more from ThreadPool than ProcessPool.

<details><summary>Answer key</summary>

1. **b**.
2. **False** – use processes for CPU.

</details>

## 11. Choosing the right model

| Workload | Best tool |
|----------|-----------|
| CPU‑bound, parallel | `multiprocessing`, Cython, NumPy |
| Many concurrent HTTP calls | `asyncio`, `aiohttp` |
| Small I/O wait + GUI program | threads |

Checklist:
1. Is work I/O‑bound? → async or threads.
2. Need >1 CPU core? → processes.
3. Need share memory? → threads with locks or manager.
4. Need millions of tasks? → async for low overhead.

```text
Decision tree:
 fetch 1000 URLs → I/O‑bound → async
 render thumbnails → CPU‑bound → process pool
 background progress bar → small I/O → thread
```

### Quick check

1. High overhead per task in threads arises from:
  a. kernel stacks   b. coroutine objects

2. True / False `asyncio` can utilise multiple CPU cores without extra processes.

<details><summary>Answer key</summary>

1. **a**.
2. **False** – still single thread.

</details>